In [ ]:
import ipywidgets as widgets
import math
import os
from IPython.display import display
from IPython.display import HTML
from IPython.display import clear_output

In [ ]:
## 0. create reusable functions for FloatText, RadioButtons, getting values

In [ ]:
# to show warning when the required values are not entered
display(HTML("""
<style>
    .red-input input {
        border: 2px solid red !important;
    }
</style>
""")
       )

In [ ]:
# to create FloatText
floattext_data = {}
def floattext_maker(title_text, label_unit_list, explanation_HTML, key_prefix):
    title = widgets.HTML(f"<div style = 'font-size: 16px'><b>{title_text}</b></div>",
                        layout = widgets.Layout(margin = '0 0 10px 0')
                        )
    
    boxes = []
    key_prefix_placeholder = placeholder_dict.get(key_prefix, {})
    
    for label, unit in label_unit_list:
        key = f"{key_prefix}_{label}"
        
        placeholder = key_prefix_placeholder.get(label.lower(), '')
        text = widgets.Text(placeholder = placeholder,
                            description = label,
                            layout = widgets.Layout(width = 'auto', min_width = '100px',
                                                    max_width = '200px'),
                            style = {'description_width': 'auto'}
                           )
        error_msg = widgets.HTML(value='', layout=widgets.Layout(margin = '2px 0 0 0'))
        def validate_input(change, error = error_msg):
            try:
                if change['new'] == '':
                    error.value = ''
                else:
                    float(change['new'])
                    error.value = ''
            except ValueError:
                error.value = "<span style='color:red; font-size:12px'>Please enter a number.</span>"
        text.observe(validate_input, names='value')
        
        floattext_data[key] = text
        floattext_unit = widgets.HTML(value = unit,
                                      layout = widgets.Layout(margin = '0 0 0 10px'))
        box = widgets.HBox([text, floattext_unit])
        boxes.append(widgets.VBox([box, error_msg]))
    explanation = widgets.HTML(value = explanation_HTML,
                              layout = widgets.Layout(margin = '10px 0 0 0')
                              )
    return widgets.VBox([title] + boxes + [explanation],
                        layout = widgets.Layout(align_items = 'flex-start')
                       )

In [ ]:
# to create RadioButtons
radiobutton_data = {}
def radiobutton_maker(title_text, options_list, label_text, explanation_HTML, key):
    title = widgets.HTML(value = f"<div style = 'font-size: 16px'><b>{title_text}</b></div>",
                        layout = widgets.Layout(margin = '0 0 10px 0')
                        )
    radiobutton = widgets.RadioButtons(options = options_list,
                                       description = '',
                                       layout = widgets.Layout(width='auto'),
                                       style = {'description_width': '0px'}
                                )
    radiobutton_data[key] = radiobutton
    _style = widgets.HTML(
        """
        <style>
        .widget-radio-box {
            display: flex !important;
            flex-direction: row !important;
            flex-wrap: nowrap !important;
            overflow: hidden !important;
            width: auto !important;
        }
        .widget-radio-box label {
            margin: 5px 10px 5px 0 !important;
            padding-left: 10px !important;
            white-space: nowrap !important;
        }
        </style>
        """,
        layout=widgets.Layout(display='none')
    )
    explanation = widgets.HTML(value = explanation_HTML,
                              layout = widgets.Layout(margin = '10px 0 0 0')
                              )
    explanation.layout.display = 'none'

    toggle_button = widgets.Button(
        description='▶ open explanation',
        layout=widgets.Layout(width='150px')
    )
    def toggle_explanation(btn):
        if explanation.layout.display == 'none':
            explanation.layout.display = 'block'
            btn.description = '▼ close explanation'
        else:
            explanation.layout.display = 'none'
            btn.description = '▶ open explanation'
    toggle_button.on_click(toggle_explanation)

    radiobutton_row = widgets.HBox([radiobutton],
        layout=widgets.Layout(
            display='flex',
            flex_flow='row',
            width='100%',
            min_width='800px',
            max_width='1000px'
        )
    )
    
    title_toggle_box = widgets.HBox([title, radiobutton],
                                    layout = widgets.Layout(align_items = 'flex-start'))
    
    box = widgets.VBox([title_toggle_box, toggle_button, explanation, _style],
                       layout = widgets.Layout(align_items = 'flex-start')
                      )
    return box

In [ ]:
# to show images
# to use: 
# name = image_presenter("pic_name.png", caption = 'caption', width = 200, height = 200)
# name = image_presenter("pic_name.png", width = 200, height = 200) - no caption
def image_presenter(image_path, caption = None, width = 300, height = 400):
    with open(image_path, 'rb') as f:
        # to extract the extension
        ext = os.path.splitext(image_path)[1][1:].lower()
        
        image = widgets.Image(value = f.read(), format = ext,
                                     width = width, height = height,
                                     layout = widgets.Layout(margin = '10px 0 0 0')
                                    )
        # to add caption
        if caption:
            caption_html = widgets.HTML(value = f"<div style = 'text-align: center'>{caption}</div>")
            image_widget = widgets.VBox([image, caption_html], 
                                         layout = widgets.Layout(width = f'{width}px',
                                                                 margin = '0 0 20px 0'
                                                                )
                                        )
        else:
            image_widget = widgets.VBox([image], 
                                        layout = widgets.Layout(width = f'{width}px',
                                                                margin = '0 0 20px 0'
                                                               )
                                       )
            return image_widget
            
    return image_widget

In [ ]:
# to extract FloatText values
# to use as a variable: {key_prefix}_values["lable"], e.g. cwof_values["weight"]
def get_floattext_values(key_prefix, label_unit_list):
    floattext_values = {}
    for label, unit in label_unit_list:
        try:
            raw_value = floattext_data[f"{key_prefix}_{label}"].value
            value = float(raw_value) if raw_value.strip() != '' else None
        except (ValueError, AttributeError):
            value = Non
        floattext_values[label] = value
    return floattext_values

In [ ]:
# list of dimensions and unit
dimension_unit_list = [("weight", "[kg]"),
                       ("width", "[mm]"),
                       ("height", "[mm]"),
                       ("thickness", "[mm]"),
                       ("thickness of painting", "[mm]"),
                       ("number of cross bars", ""),
                       ("number of length bars", "")
]

In [ ]:
# dictionaries for placeholder of each group (use with key_prefix)
placeholder_dict = {
    'canvas': {
        'weight': 'enter weight',
        'width': 'e.g. 1600',
        'height': 'e.g. 2000',
        'thickness': 'e.g. 200',
        'thickness of painting': 'e.g. 3.9'
    },
    'stretcher': {
        'weight': 'enter weight',
        'width': 'e.g. 1600',
        'height': 'e.g. 2000',
        'thickness': 'e.g. 200',
        'number of cross bars': 'e.g. 2',
        'number of length bars': 'e.g. 1'
    },
    'cwf': {
        'weight': 'enter weight',
        'width': 'e.g. 2000',
        'height': 'e.g. 2400',
        'thickness': 'e.g. 600'
    }
}

In [ ]:
# to explain how to use this app
instruction_explanation = """
This application recommends a suitable backing board construction tailored to your artwork.<br>
For more accurate calculations and recommendations, <b>we strongly encourage you to fill in all input fields.</b><br>
Please note that if some measurements are unavailable and certain fields are left blank, the accuracy of the results may be reduced.<br>
<b>Fields marked with a red box are mandatory — without these, the recommendation cannot be completed.</b><br>
<br>
Once all inputs have been completed, the "Results" tab will provide links to purchase the recommended materials,<br>
along with a downloadable PDF containing detailed instructions on how to attach them.<br>
After purchasing all the recommended materials, simply follow the steps in the PDF to complete your backing board construction.
"""
instruction_box = widgets.HTML(instruction_explanation)

In [ ]:
# dimensions of canvas without outer frame(key = "canvas") - 4 elements in list [0 4]
canvas_explanation = """
<small>
<b>For improved accuracy, entering the measured thickness is recommended.</b><br>
If the thickness of painting is not provided, the base thickness is estimated based on the presence and number of lining layers.<br>
Without lining (0 layers), it is set to 2–2.5 mm.<br>
With lining, the base thickness is set to 2–3 times the value with lining.<br>
</small>
"""
canvas_image = image_presenter("example_foto1.png", caption = 'example',
                               width = 300, height = 500
                               )
canvas_image_box = widgets.Box([canvas_image],
                               layout = widgets.Layout(display = 'flex', align_items = 'flex-end')
                               )
canvas_float_box = floattext_maker("dimension of canvas without outer frame", dimension_unit_list[:5],
                                   canvas_explanation, "canvas"
                                  )
canvas_box = widgets.VBox([canvas_float_box, canvas_image_box])

In [ ]:
# dimensions of stretcher(key = "stretcher") - 6 elements in list [0 3] + [5 6]
stretcher_explanation = """
<small>
description needed
</small>
"""
stretcher_box = floattext_maker("dimension of stretcher", dimension_unit_list[:4] + dimension_unit_list[5:],
                                stretcher_explanation, "stretcher"
                                )

In [ ]:
# dimensions of canvas with outer frame(key = "cwf") - 4 elements in list [0 3]
cwf_explanation = """
<small>
description needed
</small>
"""
cwf_image = image_presenter("example_foto2.png", caption = 'example',
                             width = 300, height = 500
                            )
cwf_image_box = widgets.Box([cwf_image],
                             layout = widgets.Layout(display = 'flex', align_items = 'flex-end')
                             )
cwf_float_box = floattext_maker("dimension of canvas with outer frame", dimension_unit_list[:4],
                                cwf_explanation, "cwf"
                                )
cwf_box = widgets.VBox([cwf_float_box, cwf_image_box])

In [ ]:
# toggle to choose - with yes(frequency knwon) or no(freqeuncy unknown)
f_toggle_title = widgets.HTML(
    value = f"<div style = 'font-size: 16px; margin: 20px 0 0 0'><b>I know the first natural frequency: </b></div>"
)
f_toggle = widgets.ToggleButtons(options = [('yes', True), ('no', False)],
                                 description = '', disabled = False
                                )
f_toggle_box = widgets.VBox([f_toggle_title, f_toggle])

In [ ]:
## 2. frequency known question - ask wheter the first natural frequency is aready known

In [ ]:
# if yes, enter the known first natural frequency
f_known = widgets.FloatText(description = 'first natural frequency:',
                            layout = widgets.Layout(width = 'auto',
                                                             min_width = '100px',
                                                             max_width = '200px'),
                            style = {'description_width': 'auto'},
                           )
f_known_unit = widgets.Label('[Hz]',
                       layout = widgets.Layout(margin = '0 0 0 10px')
                            )

In [ ]:
# input boxes for knwon frequency
f_known_box = widgets.HBox([f_known, f_known_unit])

In [ ]:
# if no, go to calculation part
f_unknown_guidance = widgets.HTML("""<div style = 'margin: 20px 0 0 0'>
The first natural frequency will be calculated based on the following parameters:
</div>
"""
                                 )

In [ ]:
# but first, enter following parameters - a) pretension
pretension_explanation = """
<small>
<b>how much is the canvas stretched?</b>
<ul style = "padding-left: 15px; list-style-type: disc;">
    <li>low:
    <ul style = "padding-left: 15px">
        <li>Canvas is limp</li>
        <li>Surface moves easily visible while painting is moved even slightly</li>
        <li>Surface sags visibly or forms a deformation at lower edge of canvas</li>
        <li>When touching it, you can clearly feel lots of space for movement 
            and/or canvas can be easily moved</li>
    </ul></li>
    <li>low-medium:
    <ul style = "padding-left: 15px">
        <li>Canvas is stretched irregularly</li>
        <li>Canvas has both areas that can be assigned to low and medium tension</li>
    </ul></li>
    <li>medium (ideal):
    <ul style = "padding-left: 15px">
        <li>Ideal pretension for a canvas painting</li>
        <li>Tension is low enough to allow necessary evasive movements during handling/
            transport, but still high enough to protect the paint layers from stronger deformations</li>
        <li>The movement during handling/transport is only slightly visible</li>
        <li>When touching it, there is a little space for movement if necessary, 
            but it is sufficiently under tension for carrying the weight of the paint 
            layers and forms a flat canvas surface</li>
    </ul></li>
    <li>medium-high:
    <ul style = "padding-left: 15px">
        <li>Canvas is stretched irregularly</li>
        <li>Canvas has  both areas that can be assigned to medium and high tension</li>
    </ul></li>
    <li>high:
    <ul style = "padding-left: 15px">
        <li>Canvas pretension is too high for painting</li>
        <li>Canvas is stretched unnecessarily far and will lead to irreversible 
            overstretching/wearing of the canvas over a long period of time</li>
        <li>When touched, canvas surface will feel like a drum</li>
    </ul></li>
</ul>
</small>
"""
pretension_box = radiobutton_maker(
    "pretension", ["low", "low-medium", "medium (ideal)", "medium-high", "high"],
    "pretension", pretension_explanation, "pretension"
)

In [ ]:
# b) lining
lining_explanation = """
<small>
<b>how many lining layers are on the painting?</b>
<ul style = "padding-left: 15px">
    <li>0: The painting is not lined - there is no lining on the painting</li>
    <li>1: description needed</li>
    <li>2: description needed</li>
    <li>3: description needed</li>
</ul>
</small>
"""
lining_box = radiobutton_maker(
    "lining", ["0", "1", "2", "3"],
    "lining", lining_explanation, "lining"
)

In [ ]:
# c) deformation
deformation_explanation = """
<small>
<b>How much is the canvas deformed?</b>
<ul style = "padding-left: 15px">
    <li>not at all: description needed</li>
    <li>medium: description needed</li>
    <li>heavy: description needed</li>
</ul>
</small>
"""
deformation_box = radiobutton_maker(
    "deformation", ["not at all", "medium", "heavy"],
    "deformation", deformation_explanation, "deformation"
)

In [ ]:
# d) impasto-painted surface
impasto_explanation = """
<small>
<b>how much surfaces are impasto-painted?</b>
<ul style = "padding-left: 15px">
    <li>0%: there is no impasto-painted surface on canvas</li>
    <li>10%: description needed</li>
    <li>20%: description needed</li>
    <li>50%: description needed</li>
</ul>
</small>
"""
impasto_box = radiobutton_maker(
    "impasto-painted surface", ["0%", "10%", "20%", "50%"],
    "impasto-painted surface", impasto_explanation, "impasto"
)

In [ ]:
# input boxes for unknown frequency leading to calculation
f_unknown_box = widgets.VBox([f_unknown_guidance, pretension_box,
                              lining_box, deformation_box,impasto_box]
                            )

In [ ]:
## 3. output area
# calculation output
cal_output = widgets.Output()
# warning output
warning_output = widgets.Output()
# display output
decision_output = widgets.Output()
# result output
result_output = widgets.Output()

In [ ]:
# tabs of input boxes for dimension
input_tab_titles = ["instruction", "canvas", "stretcher", "canvas with frame", "1st natural frequency"]
input_tab_children = [instruction_box, canvas_box, stretcher_box, cwf_box, widgets.VBox([f_toggle_box, decision_output, cal_output])]
input_tab = widgets.Tab(children = input_tab_children)
for i in range(len(input_tab_titles)):
    input_tab.set_title(i, input_tab_titles[i])

In [ ]:
# summarize results and show the result tab
result_tab = widgets.Tab(children=[result_output])
result_tab.set_title(0, 'Result')
result_tab.layout.display = 'block'
result_state = {"f": None, "mode": None}
def result_summary(f = None, mode = None, inputs_only = False):
    with result_output:
        clear_output()
        # show input values
        def show_group(title, key_prefix, label_unit_list):
            values = get_floattext_values(key_prefix, label_unit_list)
            if all(values[label] is None for label, _ in label_unit_list):
                return
            html = f"<div style = 'margin: 10px 0 0 0;'>{title}<ul style = 'margin: 0 0 0 10px;'>"
            for label, unit in label_unit_list:
                value = values.get(label)
                if value is not None:
                    html += f"<li>{label.replace('_', ' ')}: {value} {unit}</li>"
            html += "</ul></div>"
            display(HTML(html))
        show_group("Canvas", "canvas", dimension_unit_list[:5])
        show_group("Stretcher", "stretcher", dimension_unit_list[:4] + dimension_unit_list[5:])
        show_group("Canvas with outer frame", "cwf", dimension_unit_list[:4])
        
        # show frequency
        if f is not None:
            label = "Entered" if mode == 'entered' else "Calculated"
            display(widgets.HTML(
                f"<b>{label} first natural frequency:</b> {f:} Hz"
            ))
        else:
            display(HTML("<div style = 'margin: 10px 0 0 0;'>No frequency entered/calculated yet.</div>"))

In [ ]:
## 4. calculation function
def f_calculation():
    with cal_output:
        clear_output()

        # common values for calculation and later recommendation
        canvas_values = get_floattext_values("canvas", dimension_unit_list[:5])

        # show warning, in case the required values are not entered
        missing_fields = []
        for key in ["width", "height"]:
            val = canvas_values[key]
            field = floattext_data[f"canvas_{key}"]
            if val in (None, 0):
                field.add_class("red-input")
                missing_fields.append(key)
            else:
                field.remove_class("red-input")
        with warning_output:
            clear_output()
            if missing_fields:
                fields = ", ".join(missing_fields)
                display(HTML(f"<div style='color:red'>Please enter all red-boxed values.</div>"))

        # initializaion
        f = None
        
        # if yes:
        if f_toggle.value == True:
            if f_known.value is None:
                print("Please enter the first natural frequency.")
                return
            f = f_known.value
            
        # if no:
        else:
            # mapping factors through values user gave
            pretension_value = radiobutton_data["pretension"].value
            lining_value = radiobutton_data["lining"].value
            deformation_value = radiobutton_data["deformation"].value
            impasto_value = radiobutton_data["impasto"].value
            
            TensionFactors = {"low": 0.7, "low-medium": 0.8, "medium (ideal)": 1.0,
                                  "medium-high": 1.1, "high": 1.2}
            LiningFactors = {"0": 1, "1": 1.1, "2": 1.2, "3": 1.3}
            DeformFactors = {"not at all": 1, "medium": 1.1,
                                   "heavy": 1.2}
            ImpastoFactors = {"0%": 1, "10%": 1.1, "20%": 1.4,
                               "50%": 2}

            TensionFactor = TensionFactors.get(pretension_value, None)
            LiningFactor = LiningFactors.get(lining_value, None)
            DeformFactor = DeformFactors.get(deformation_value, None)
            ImpastoFactor = ImpastoFactors.get(impasto_value, None)

            # base value of parameters, later adjusted
            base_thickness_average = 2
            base_Emodule = 400
            base_density = 400

            # adjusting & sub-calculations
            if canvas_values["thickness of painting"] is not None and canvas_values["thickness of painting"] != 0:
                canvas_values["thickness of painting"] = canvas_values["thickness of painting"]
            else:
                if lining_value == "0":
                    canvas_values["thickness of painting"] = base_thickness_average
                else:
                    canvas_values["thickness of painting"] = base_thickness_average*2.5
            Emodule = base_Emodule*TensionFactor*LiningFactor*DeformFactor
            poisson = 0.3
            density = base_density*ImpastoFactor
            bending_stiffness = round((Emodule*canvas_values["thickness of painting"]**3)/(12*(1-poisson**2)), 2)

            # final result of calculation
            if canvas_values["width"] is None or canvas_values["height"] is None:
                f = None
            else:
                f = round((math.pi/2)*
                          (math.sqrt(bending_stiffness/(density*canvas_values["thickness of painting"])))*
                          (1/(canvas_values["width"]/1000)**2 + 1/(canvas_values["height"]/1000)**2),
                          1)

            # to display the calculated frequency and according recommendation
            if f is not None:
                f_unknown_result = f"""
                <div style = 'margin: 20px 0 20px 0'>
                <b>Calculated first natural frequency:</b> {f: .1f} Hz
                </div>
                """
                display(HTML(f_unknown_result))
                update_result_state(f, 'calculated')
            else:
                f_unknown_result = f"""
                <div style = 'margin: 20px 0 20px 0'>
                <b>Calculated first natural frequency:</b> 
                <small>(e.g. 0.5 Hz)</small> 
                <i>wating for input...</i>
                </div>
                """
                display(HTML(f_unknown_result))
                result_summary(f=None, mode=None)

In [ ]:
## 6. update function
# for result
def update_result_state(f, mode):
    result_state["f"] = f
    result_state["mode"] = mode
    result_summary(f, mode)
    
# to show right widgets after choosing toggle
def update_display(change = None):
    decision_output.clear_output()
    with decision_output:
        if f_toggle.value:
            display(f_known_box)
            if change and change.get("new") == True:
                result_state["f"] = None
                result_state["mode"] = None
                result_summary(f=None, mode=None)
            else:
                result_summary(**result_state)
        else:
            display(f_unknown_box)
            f_calculation()
            if result_state["f"] is not None:
                result_summary(**result_state)

In [ ]:
## 7. for real-time according to the changes of value
# observe changes of toggle (yes <-> no)
f_toggle.observe(update_display, names='value')
# observe changes of given frequency
f_known.observe(
    lambda change: (
        f_calculation(), update_result_state(change['new'], 'entered')
                   )
    if f_toggle.value and change['new'] > 0 else None,
    names = 'value'
)
# observe changes of parameters when the 'no' is selected
for key in ["pretension", "lining", "deformation", "impasto"]:
     radiobutton_data[key].observe(lambda change: f_calculation(), names='value')
for key_prefix in ["canvas", "stretcher", "cwf"]:
    for label, _ in dimension_unit_list:
        key = f"{key_prefix}_{label}"
        if key in floattext_data:
            floattext_data[key].observe(lambda change: (f_calculation(), result_summary(inputs_only = True)),
                                        names='value'
                                       )

In [ ]:
update_display()

In [ ]:
## 8. display everything together as one flow
full_ui = widgets.VBox([input_tab,
                        warning_output,
                        result_tab
                       ]
                      )
display(full_ui)